<p style="font-size:24px">
<b> To lauch the notebook select "Run All" from the "Cell" menu above. </b>
<p style="font-size:36px">
<b> Eye Diagrams </b>
<p style="font-size:18px">
This notebook demonstrates how amplitude noise, bandwidth limitations, and timing jitter lead to errors in determining the value of a transmitted bit, and how an eye diagram provides a visual illustration of the contributions to errors.

<p style="font-size:24px">
<b> Noise Sources </b>
<p style="font-size:18px">
One of the most important contributions to errors in the detection of optical signals is thermal noise in the optical receiver. Thermal noise arises from fluctuations in the current through a resitive load. The noise current has a Gaussian distribution that is characterized by its standard deviation, given by the expression
<p style="font-size:18px">
$${\sigma _T} = \sqrt {\frac{{4{k_B}T}}{{{R_L}}}\Delta f}, $$
where $k_B$ is Boltzman's constant, T is the temperature, $R_L$ is the resistance of the load that is used to generate a voltage from the photodetector current, and $\Delta f$ is the bandwidth of the optical receiver. Current variation leads to a variation in the voltage across the load resistor and to a variation in the amplitude of the measured voltage.
<p style="font-size:18px">
Thermal noise constibutes equally to variation in the amplitude for transmitted "ones" and "zeros". More generally there are additional noise sources that contibute in different amounts to ones and zeros and that are characterized by two different standard deviations ${\sigma _1}$ and ${\sigma _0}$. Shot noise is an example is example of this latter type of noise. Shot noise is always present for the detection of optical signals, but it can be neglected if the signal levels are not small. In this notebook we will use ${\sigma _1}$ = ${\sigma _0}$ = ${\sigma _T}$.

<p style="font-size:24px">
<b>  Q-Factor and Bit Error Rate </b>
<p style="font-size:18px">
When describing noise for optical recievers, it is common to define a quantity called the Q-Factor, which is similar to signal-to-noise ratio but is the appropriate quantity to use for detection of both one and zero levels. The Q-Factor is given by the expression
<p style="font-size:18px">
$$Q \equiv \frac{{{I_1} - {I_0}}}{{{\sigma _0} + {\sigma _1}}},$$
<p style="font-size:18px">
where $I_1$ and $I_0$ are the average current generated by a transmitted one and zero, respectively. What we really want to know about our optical detection system is its bit error rate (BER, the fraction of bits incorrectly detected) that is related to Q-Factor  by the expression
<p style="font-size:18px">
$$BER = \frac{1}{2}erfc\left( {\frac{Q}{{\sqrt 2 }}} \right),$$
where erfc is the complementary error function
<p style="font-size:18px">

<p style="font-size:24px">
<b>  Visualizing Measurment Error </b>
<p style="font-size:18px">
The graphs below are for an optical receiver that has an optical power to current conversion factor (responsivity) of 0.5, a resistive load of 50 ohm, and a voltage gain of 10x. The data rate is 2.5 gigabits per second. The bandwidth of the receiver is taken to be 20 GHz, a value significantly larger than what is required to resolve data at this rate, in order to increase the noise level and make its effects easier to see. The slider lets you examine the effects of noise for incoming optical signals with various amounts of average power. The bandwidth of the receiver is taken to be 20 GHz 
<p style="font-size:18px">
The graph on the left shows a voltage signal that represents the bit sequence 0, 1, 0, as indicated in the graph, with the labels located in the center of the bit periods. The graph on the right shows the distributions of voltage measurements. More precisely, the oragnme curves in the graph on the rigth show the probability densities (per volt) of the measured voltage. The histograms indicate the number of measured values placed each of 100 bins, for 1000 measurments. The values in the histogram have been normalized so as better fit the oragnge curves. A transmitted bit is determined to be a one if a measurment is above the voltage threshold, and a zero if a measurement is below the threshold. Signals are measured at the middle of a bit period. There is always a non-zero probability that a signal will be on the wrong side of the threshold level during a measurement, resulting in an incorrect translation of a signal into a bit.

<p style="font-size:24px">
<b> Exploration </b>
<p style="font-size:18px">
Explore how signal power affects the bit-error rate with the interactive graphs below
<p style="font-size:18px">
Suggestion: Adjust the average received power to achieve a bit error rate of around 1 x $10^{-4}$. Can you see the signal drop below the threshold value in the left-hand chart or overlap of the probability distributions in the right-hand chart? Why or why not?
<p style="font-size:18px">
Suggestion: Adjust the singal slider so that the signal level occasionally falls below the threshold level and the probability distributions overlap. What is the bit error rate in this case?

In [1]:
%run load_modules.ipynb

Loading BokehJS ...

In [2]:
%run err_dist_chart.ipynb

interactive(children=(FloatSlider(value=10.0, description='Pave (μW)', max=35.0), Output()), _dom_classes=('wi…

<p style="font-size:24px">
<b> Finite Bandwidth and Signal Jitter </b>
<p style="font-size:18px">
An optical receiver with a very high bandwidth also incorporates more detection noise into the photodetection process. This can be seen by examing the expression for the size of thermal noise that is given above. For this reason, the bandwidth of optical receivers is typically adjusted so as to be just large enough to be able to resolve incoming bits of ones and zeros. The bandwidth for an optical receiver for detecting a bit stream at 2.5 gigabits per second will be on the order of 2.5 GHz. Note that finite bandwidth "rounds" signals and reduces the "flat" regions in the middle of bit periods where measurments take place. When combined with signal jitter (a random error in the timing of the transitions between bit periods), this can also lead to increased error rates.
    
<p style="font-size:24px">
<b> Exploration </b>
<p style="font-size:18px">
Explore how combinations of bandwidth and timing jitter distort signals. Just below, the curve on the left shows the frequency components for a square wave signal and added thermal noise contribution. The graph also shows the filtering reponse of an optical receiver with a 3dB bandwdith set by a slider. The graph on the right shows the filtered/bandwidth-limited signal.

<p style="font-size:18px">
Suggesion: Assume that measurements are made at the center of the bit periods. Find a combination of bandwidth, average signal power, and jitter  for which you can observe that an error would be made in the measurment of a bit value.

In [3]:
%run bandwidth_jitter_chart.ipynb

interactive(children=(FloatSlider(value=10.0, description='Bandwidth (GHz)', max=40.01, min=1.6, style=SliderS…

## Building an Eye Diagram
<p style="font-size:18px">
A measurment of bit error rate provides a single, quantitative value that provides a valuable, overall assessment of the peformance of an optical link. An eye diagram is a visual tool that can be used to quickly produce a qualtitative assessment of the various contributions to the bit error rate, including themal noise, finite bandwidth, and timing jitter. An eye diagram is created by overlapping multiple signal traces, each carrying a separate random bit pattern. The traces are typically two bit periods long, with half a bit period followed by a full bit period, followed by half a bit period.

### Exploration
<p style="font-size:18px">
The graph just below shows how an eye diagram is built up by drawing multiple segments of bit stream. Click on the "Trace" button to draw a signal curve on the graph. The random bit pattern will be one of eight - 000. 001, 010, 011, 100, 101, 110, or 111. Up to 21 curves can be drawn on the graph. The "Reset" button clears the graph and grabs the new parameters if you have adjusted the sliders in the second figure above.

<p style="font-size:18px">
Suggestion: Find the minimum number of signal traces needed to create an eye diagram that shows both the vertical and horizontal undertainty in the signal values.

In [4]:
%run eye_diagram_chart.ipynb